In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import os
from tqdm import tqdm
from astropy.io import fits
from astropy.wcs import WCS
from astropy.wcs.utils import pixel_to_skycoord, skycoord_to_pixel
from astropy.coordinates import SkyCoord
from astropy.nddata import Cutout2D
from scipy.ndimage.filters import gaussian_filter
tqdm.pandas()
import cmocean
import matplotlib as mpl

C:\Users\Matt\.conda\envs\tf2-gpu\lib\site-packages\tqdm\std.py:706: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
from reproject.mosaicking import find_optimal_celestial_wcs
from reproject import reproject_interp
from reproject.mosaicking import reproject_and_coadd

In [3]:
PATH_RAW = 'D:/Workshop/PhD/Data/ViaLactea/PPMAP_Results/'
PATH_64x85 = 'D:/Workshop/PhD/Data/ViaLactea/CNN_Results/64px_rejigged_fulltile/'
PATH_32x17 = 'D:/Workshop/PhD/Data/ViaLactea/CNN_Results/32px_rejigged_fulltile_64pxwindow/'
PATH_32x85 = 'D:/Workshop/PhD/Data/ViaLactea/CNN_Results/32px_rejigged_fulltile/'

In [4]:
no_files = len(os.listdir(PATH_32x17+'DataCubes'))
no_files

163

In [ ]:
x = 0.03
y = 0.93
sigma = 2

plt.rcParams.update({'font.size': 12})
f,ax = plt.subplots(3,3, figsize=(15,15))

# H
df = pd.DataFrame(columns = ['H_32x85','H_64x85','H_32x17'])

for i in tqdm(range(no_files)):
    cube_64x85 = pickle.load(open(f'{PATH_64x85}/DataCubes/{os.listdir(PATH_64x85+"DataCubes")[i]}','rb'))
    cube_32x85 = pickle.load(open(f'{PATH_32x85}/DataCubes/{os.listdir(PATH_32x85+"DataCubes")[i]}','rb'))
    cube_32x17 = pickle.load(open(f'{PATH_32x17}/DataCubes/{os.listdir(PATH_32x17+"DataCubes")[i]}','rb'))

    temp_df = pd.DataFrame()
    temp_df['H_64x85'] = np.pad(cube_64x85[1],((32,31),(32,31)), 'constant', constant_values = np.nan).flatten()
    temp_df['H_32x85'] = np.pad(cube_32x85[1],((16,15),(16,15)), 'constant', constant_values = np.nan).flatten()
    temp_df['H_32x17'] = np.pad(cube_32x17[1],((32,31),(32,31)), 'constant', constant_values = np.nan).flatten()
    temp_df = temp_df.dropna()
    df = df.append(temp_df)

ax[0,2].hist(df['H_64x85'], bins=100, facecolor='dimgrey')
ax[0,2].set_xlabel(r'$\mathcal{H}_{64\times 8.5}$')
ax[0,2].text(x,y, '(c)', transform=ax[0,2].transAxes,bbox = dict(facecolor='white'))
ax[0,2].set_xlim([0,1.25])

ax[0,1].hist(df['H_32x17'], bins=100, facecolor='dimgrey')
ax[0,1].set_xlabel(r'$\mathcal{H}_{32\times 17}$')
ax[0,1].text(x,y, '(b)', transform=ax[0,1].transAxes,bbox = dict(facecolor='white'))
ax[0,1].set_xlim([0,1.25])

ax[0,0].hist(df['H_32x85'], bins=100, facecolor='dimgrey')
ax[0,0].set_xlabel(r'$\mathcal{H}_{32\times 8.5}$')
ax[0,0].text(x,y, '(a)', transform=ax[0,0].transAxes,bbox = dict(facecolor='white'))
ax[0,0].set_xlim([0,1.25])

# S

df = pd.DataFrame(columns = ['S_32x85','S_64x85','S_32x17'])

for i in tqdm(range(no_files)):
    cube_64x85 = pickle.load(open(f'{PATH_64x85}/DataCubes/{os.listdir(PATH_64x85+"DataCubes")[i]}','rb'))
    cube_32x85 = pickle.load(open(f'{PATH_32x85}/DataCubes/{os.listdir(PATH_32x85+"DataCubes")[i]}','rb'))
    cube_32x17 = pickle.load(open(f'{PATH_32x17}/DataCubes/{os.listdir(PATH_32x17+"DataCubes")[i]}','rb'))

    temp_df = pd.DataFrame()
    temp_df['S_64x85'] = np.pad(cube_64x85[2],((32,31),(32,31)), 'constant', constant_values = np.nan).flatten()
    temp_df['S_32x85'] = np.pad(cube_32x85[2],((16,15),(16,15)), 'constant', constant_values = np.nan).flatten()
    temp_df['S_32x17'] = np.pad(cube_32x17[2],((32,31),(32,31)), 'constant', constant_values = np.nan).flatten()
    temp_df = temp_df.dropna()
    df = df.append(temp_df)
    
ax[1,2].hist(df['S_64x85'], bins=100, facecolor='dimgrey')
ax[1,2].set_xlabel(r'$\mathcal{S}_{64\times 8.5}$')
ax[1,2].text(x,y, '(f)', transform=ax[1,2].transAxes,bbox = dict(facecolor='white'))
ax[1,2].set_xlim([0,3.25])

ax[1,1].hist(df['S_32x17'], bins=100, facecolor='dimgrey')
ax[1,1].set_xlabel(r'$\mathcal{S}_{32\times 17}$')
ax[1,1].text(x,y, '(e)', transform=ax[1,1].transAxes,bbox = dict(facecolor='white'))
ax[1,1].set_xlim([0,3.25])

ax[1,0].hist(df['S_32x85'], bins=100, facecolor='dimgrey')
ax[1,0].set_xlabel(r'$\mathcal{S}_{32\times 8.5}$')
ax[1,0].text(x,y, '(d)', transform=ax[1,0].transAxes,bbox = dict(facecolor='white'))
ax[1,0].set_xlim([0,3.25])

#HvS

df = pd.DataFrame(columns = ['H_64x85','S_64x85'])
for i in tqdm(range(no_files)):
    
    cube_64x85 = pickle.load(open(f'{PATH_64x85}/DataCubes/{os.listdir(PATH_64x85+"DataCubes")[i]}','rb'))
    
    temp_df = pd.DataFrame()
    temp_df['H_64x85'] = np.pad(cube_64x85[1],((32,31),(32,31)), 'constant', constant_values = np.nan).flatten()
    temp_df['S_64x85'] = np.pad(cube_64x85[2],((32,31),(32,31)), 'constant', constant_values = np.nan).flatten()
    temp_df = temp_df.dropna()
    df = df.append(temp_df)
    
H_min = df['H_64x85'].min()
H_max = df['H_64x85'].max()

S_max = df['S_64x85'].max()
S_min = df['S_64x85'].min()
    
S_bins = np.linspace(S_min, S_max, 201)
H_bins = np.linspace(H_min, H_max, 201)

counts, xbins, ybins, image = ax[2,2].hist2d(df['S_64x85'],df['H_64x85'],bins=[S_bins,H_bins], cmap=cmocean.cm.thermal, norm=mpl.colors.LogNorm())

contours = ax[2,2].contour(gaussian_filter(counts.transpose(),sigma), 
                extent = [xbins.min(),xbins.max(),ybins.min(),ybins.max()],
                levels = [1000,10000,70000],
                           colors='k'
               )

ax[2,2].set_xlabel(r'$\mathcal{S}_{64\times 8.5}$')
ax[2,2].set_ylabel(r'$\mathcal{H}_{64\times 8.5}$')
ax[2,2].text(x,y, '(i)', transform=ax[2,2].transAxes,bbox = dict(facecolor='white'))
ax[2,2].set_xlim([0,3.25])
ax[2,2].set_ylim([0,1.25])

df = pd.DataFrame(columns = ['H_32x17','S_32x17'])
for i in tqdm(range(no_files)):
    
    cube_32x17 = pickle.load(open(f'{PATH_32x17}/DataCubes/{os.listdir(PATH_32x17+"DataCubes")[i]}','rb'))

    temp_df = pd.DataFrame()
    temp_df['H_32x17'] = np.pad(cube_32x17[1],((32,31),(32,31)), 'constant', constant_values = np.nan).flatten()
    temp_df['S_32x17'] = np.pad(cube_32x17[2],((32,31),(32,31)), 'constant', constant_values = np.nan).flatten()
    temp_df = temp_df.dropna()
    df = df.append(temp_df)
    
counts, xbins, ybins, image = ax[2,1].hist2d(df['S_32x17'],df['H_32x17'],
                                             bins=[S_bins,H_bins], 
                                             cmap=cmocean.cm.thermal, 
                                             norm=mpl.colors.LogNorm())
contours = ax[2,1].contour(gaussian_filter(counts.transpose(),sigma), 
                extent = [xbins.min(),xbins.max(),ybins.min(),ybins.max()],
                levels = [1000,10000,70000],
                           colors='k'
               )

ax[2,1].set_xlabel(r'$\mathcal{S}_{32\times 17}$')
ax[2,1].set_ylabel(r'$\mathcal{H}_{32\times 17}$')
ax[2,1].text(x,y, '(h)', transform=ax[2,1].transAxes,bbox = dict(facecolor='white'))
ax[2,1].set_xlim([0,3.25])
ax[2,1].set_ylim([0,1.25])

df = pd.DataFrame(columns = ['H_32x85','S_32x85'])
for i in tqdm(range(no_files)):
    
    cube_32x85 = pickle.load(open(f'{PATH_32x85}/DataCubes/{os.listdir(PATH_32x85+"DataCubes")[i]}','rb'))

    temp_df = pd.DataFrame()
    temp_df['H_32x85'] = np.pad(cube_32x85[1],((16,15),(16,15)), 'constant', constant_values = np.nan).flatten()
    temp_df['S_32x85'] = np.pad(cube_32x85[2],((16,15),(16,15)), 'constant', constant_values = np.nan).flatten()
    temp_df = temp_df.dropna()
    df = df.append(temp_df)

counts, xbins, ybins, image = ax[2,0].hist2d(df['S_32x85'],df['H_32x85'],bins=[S_bins,H_bins], cmap=cmocean.cm.thermal, norm=mpl.colors.LogNorm())

contours = ax[2,0].contour(gaussian_filter(counts.transpose(),sigma), 
                extent = [xbins.min(),xbins.max(),ybins.min(),ybins.max()],
                levels = [1000,10000,70000],
                           colors='k'
               )

ax[2,0].set_xlabel(r'$\mathcal{S}_{32\times 8.5}$')
ax[2,0].set_ylabel(r'$\mathcal{H}_{32\times 8.5}$')
ax[2,0].text(x,y, '(g)', transform=ax[2,0].transAxes,bbox = dict(facecolor='white'))
ax[2,0].set_xlim([0,3.25])
ax[2,0].set_ylim([0,1.25])

plt.savefig('D:\Workshop\PhD\Papers\HiGal paper\Images/DistributionsNew.png',bbox_inches="tight")
plt.show()

100%|████████████████████████████████████████████████████████████████████████████████| 163/163 [07:54<00:00,  2.91s/it]
